In [1]:
import numpy as np 

data_path = "/data/qgong/data/AE/CICDDoS2019/Data/cicddos"

X_train = np.load(data_path+"/X_train.npy")
y_train = np.load(data_path+"/y_train.npy")
X_test = np.load(data_path+"/X_test.npy")
y_test = np.load(data_path+"/y_test.npy")

X_adv = np.load(data_path+"/Adv/Adv_X_test.npy")
y_adv = np.copy(y_test)

print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_adv))

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_adv.shape, y_adv.shape

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


((490783, 78), (490783, 78), (490783,), (490783,), (490783, 78), (490783,))

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score


# Parameters
input_dim = 78  # Replace with the number of features in your tabular dataset
output_dim = 11  # Replace with the number of classes


from sklearn.metrics import precision_score, recall_score, f1_score

# Define a more practical classifier architecture
def create_classifier(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(output_dim, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


pretrained_classifier = create_classifier(input_dim, output_dim)
pretrained_classifier.load_weights('classifier.weights.h5')
# pretrained_classifier.compile()

pretrained_classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Verify the loaded model works
loss, accuracy = pretrained_classifier.evaluate(X_test, y_test, verbose=0)

# Predict class probabilities and get the predicted classes
y_pred_prob = pretrained_classifier.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)

# Compute precision, recall, and F1 score using scikit-learn
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for multi-class
recall = recall_score(y_test, y_pred, average='weighted')        # Weighted for multi-class
f1 = f1_score(y_test, y_pred, average='weighted')                # Weighted for multi-class

# Print the metrics
print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")

2024-12-04 13:50:38.562253: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 13:50:38.586518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 13:50:38.586541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 13:50:38.586566: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 13:50:38.592035: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 13:50:38.592448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

15337/15337 [==============================] - 8s 529us/step
Accuracy: 0.890605
Precision: 0.909746
Recall: 0.890605
F1 Score: 0.887729


In [ ]:
from alibi_detect.ad import AdversarialAE
from alibi_detect.utils.saving import save_detector

# Define encoder and decoder (same as before or custom)
encoder_net = tf.keras.Sequential(
    [
        Input(shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(32)  # Latent representation
    ]
)

decoder_net = tf.keras.Sequential(
    [
        Input(shape=(32,)),
        Dense(64, activation='relu'),
        Dense(input_dim)  # Reconstruct original input
    ]
)

# Initialize AdvAE with the pre-trained classifier
ad = AdversarialAE(
    encoder_net=encoder_net,
    decoder_net=decoder_net,
    model=pretrained_classifier,
    temperature=0.5
)

# Train the AdvAE
ad.fit(X_train, epochs=3, batch_size=64, verbose=True)

# Save the trained AdvAE detector
save_detector(ad, 'adv_ae_detector_with_pretrained_classifier')

No threshold level set. Need to infer threshold using `infer_threshold`.


3464/7669 [.] - ETA: 1:02 - loss_ma: 0.0909

KeyboardInterrupt: 